In [7]:
from xlrd import open_workbook  # para abrir el excel
import numpy as np

from netCDF4 import Dataset, num2date, date2index  
#from osgeo import gdal
from matplotlib.mlab import griddata
#from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline



ImportError: libmfhdf.so.0: cannot open shared object file: No such file or directory

In [ ]:
############## Validación datos de estaciones de superficie ###################

In [3]:
## Abro los datos de las estaciones
datos_pp = open_workbook('datos_val36hs_17AGO.xls')
sheet_names = datos_pp.sheet_names()  # List sheet names, and pull a sheet by name
xl_sheet = datos_pp.sheet_by_name(sheet_names[0])
# Lee los valores de cada columna
lat_obs = xl_sheet.col_values(1)
lon_obs = xl_sheet.col_values(2)
pp_36hs_obs = xl_sheet.col_values(3)


In [4]:
len(pp_36hs_obs)

166

In [5]:
## Abro los datos del modelo
f =Dataset('wrf_ppAcum_36hs.nc')
rainnc_36hs_wrf = f.variables['rainnc_36hs'][:]
lat_wrf = f.variables['lat'][:]
lon_wrf = f.variables['lon'][:]

NameError: name 'Dataset' is not defined

In [17]:
# Paso los datos del modelo a la retícula de las estaciones (sugerencia de Juan)
# uso el método de interpolación nearest neighbord

In [47]:
pp_wrf_interpolada = griddata((lon_wrf.ravel(),lat_wrf.ravel()), rainnc_36hs_wrf.ravel(), (lon_obs, lat_obs), method = 'nearest')


In [48]:
len(pp_wrf_interpolada)

166

In [51]:
## grafico
lat1 = np.arange(-55, -20, 0.036) #se interpola con una resolución de 4 km
lon1 = np.arange(-78, -50, 0.036)
lon_mesh, lat_mesh = np.meshgrid(lon1,lat1)
## Interpolate at the points in lon_mesh, lat_mesh en nn
pp_obs_mesh_nn = griddata(lon_obs, lat_obs, pp_36hs_obs, (lon_mesh, lat_mesh), method = 'nearest')
pp_wrf_mesh_nn = griddata(lon_obs, lat_obs, pp_wrf_interpolada, (lon_mesh, lat_mesh), method = 'nearest')

#http://students.mimuw.edu.pl/~pbechler/scipy_doc/generated/scipy.interpolate.griddata.html

TypeError: griddata() got multiple values for keyword argument 'method'

In [ ]:
#Parámetros de la proyección Lambert
lon_0 = -63.  
lat_0 = -31.
lat_1 = -31.
lat_2 = -31.

## Limites del campo
minlat = -45
maxlat = -20
minlon = -70
maxlon = -50

## Tipo de proyeccion en el mapa: Lambert

lamb = Basemap(projection='lcc',lon_0=lon_0,lat_0=lat_0,\
              llcrnrlat=minlat,urcrnrlat=maxlat, llcrnrlon=minlon,urcrnrlon=maxlon,\
              lat_1=lat_1,lat_2=lat_2,resolution='h',area_thresh=1000.)

#cmap = plt.get_cmap('Spectral')
levels = [0.1,10,20,30,40,50,60,70]
x_mesh, y_mesh = lamb(lon_mesh, lat_mesh)

In [ ]:
fig_size[0] = 15
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size

fig = plt.figure(figsize=(12,10))
plt.subplot(1,2,1)

m = lamb.contourf(x_mesh, y_mesh, pp_obs_mesh_nn,levels = levels, cmap = my_cmap)

x,y = lamb(lon_obs, lat_obs)
smn = plt.scatter(x[1:85], y[1:85], marker= 'o', color = 'm', label = 'SMN')
bcr = plt.scatter(x[85:115], y[85:115], marker= 'o', color = 'b', label = 'BCR')
sl = plt.scatter(x[115:156], y[115:156], marker= 'o', color = 'g', label = 'SL')
srrhh = plt.scatter(x[156:], y[156:], marker= 'o', color = 'r',  label = 'SRRHH')

lamb.drawcoastlines()
lamb.drawparallels(np.arange(-40.,-20.,5.),labels=[1,0,0,0]) # draw parallels
lamb.drawmeridians(np.arange(-80.,-45.,5.),labels=[0,0,0,1]) # draw meridians
plt.title('Datos de superficie') # add a title
plt.colorbar(m, orientation='horizontal') 

##linear##
plt.subplot(1,2,2)
ml = lamb.contourf(x_mesh, y_mesh, pp_wrf_mesh_nn,levels = levels, cmap = my_cmap)
x,y = lamb(lon_obs, lat_obs)
plt.scatter(x[1:85], y[1:85], marker= 'o', color = 'm', label = 'SMN')
plt.scatter(x[85:115], y[85:115], marker= 'o', color = 'b', label = 'BCR')
plt.scatter(x[115:156], y[115:156], marker= 'o', color = 'g', label = 'SL')
plt.scatter(x[156:], y[156:], marker= 'o', color = 'r', label = 'SRRHH')

lamb.drawcoastlines()
lamb.drawparallels(np.arange(-40.,-20.,5.),labels=[1,0,0,0]) # draw parallels
lamb.drawmeridians(np.arange(-80.,-45.,5.),labels=[0,0,0,1]) # draw meridians
plt.title('Datos de wrf') # add a title
plt.colorbar(ml,orientation='horizontal') 

plt.suptitle('Datos de pp acumulada 24 hs', size = 20)
legend = plt.legend((smn, bcr, sl, srrhh),('SMN', 'BCR', 'SL', 'SRRHH'), loc = 'best', bbox_to_anchor = (1., 1.02))
#bbox_to_anchor sirve para sacar la legenda del plot

plt.show()
#plt.figlegend((smn, bcr),('SMN', '

In [ ]:
##Validación con datos de TRMM


## Pasar WRF a la resolución de TRMM (4 km a 025º)

## VER LO DE JUAN